## Assignment 4 - Question Duplicates

<a name='0'></a>
## Overview
In this assignment, concretely you will: 

- Learn about Siamese networks
- Understand how the triplet loss works
- Understand how to evaluate accuracy
- Use cosine similarity between the model's outputted vectors
- Use the data generator to get batches of questions
- Predict using your own model

By now, you are familiar with trax and know how to make use of classes to define your model. We will start this homework by asking you to preprocess the data the same way you did in the previous assignments. After processing the data you will build a classifier that will allow you to identify whether two questions are the same or not. 
<img src = "images/meme.png" style="width:550px;height:300px;"/>


You will process the data first and then pad in a similar way you have done in the previous assignment. Your model will take in the two question embeddings, run them through an LSTM, and then compare the outputs of the two sub networks using cosine similarity. Before taking a deep dive into the model, start by importing the data set.

### 1. Importing the data

In [1]:
import os
import nltk
import trax
from trax import layers as tl
from trax.supervised import training
from trax.fastmath import numpy as fastnp
import numpy as np
import pandas as pd
import random as rnd
from trax import shapes

import w4_unittest

nltk.data.path.append('nltk_data')
rnd.seed(4)

2023-12-29 12:08:17.907890: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-29 12:08:17.907950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-29 12:08:17.909061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-29 12:08:18.951145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv('data/questions.csv')
print(f"number of questions pairs: {len(data)}")
data.head()

number of questions pairs: 404351


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
N_TRAIN = 300000
N_TEST=10*1024
data_train = data[:N_TRAIN]
data_test = data[N_TRAIN:N_TRAIN + N_TEST]

print(f"length of training set: {len(data_train)}, length of testing set: {len(data_test)}")

length of training set: 300000, length of testing set: 10240


In [4]:
is_duplicate_index = data_train[data_train['is_duplicate'] == True].index.to_list()
is_duplicate_index

[5,
 7,
 11,
 12,
 13,
 15,
 16,
 18,
 20,
 29,
 31,
 32,
 38,
 48,
 49,
 50,
 51,
 53,
 58,
 62,
 65,
 66,
 67,
 71,
 72,
 73,
 74,
 79,
 84,
 85,
 86,
 88,
 92,
 93,
 95,
 100,
 104,
 107,
 113,
 120,
 122,
 125,
 127,
 135,
 136,
 143,
 144,
 152,
 156,
 158,
 159,
 160,
 163,
 165,
 168,
 173,
 175,
 176,
 178,
 179,
 180,
 182,
 185,
 188,
 189,
 190,
 191,
 193,
 194,
 197,
 198,
 199,
 200,
 203,
 209,
 210,
 215,
 216,
 219,
 220,
 221,
 224,
 226,
 229,
 235,
 236,
 238,
 242,
 243,
 244,
 246,
 249,
 250,
 251,
 253,
 255,
 260,
 261,
 262,
 267,
 269,
 270,
 273,
 274,
 275,
 281,
 284,
 285,
 286,
 287,
 288,
 291,
 293,
 295,
 296,
 299,
 304,
 307,
 308,
 309,
 312,
 317,
 318,
 321,
 322,
 323,
 326,
 329,
 331,
 339,
 341,
 346,
 347,
 348,
 349,
 350,
 353,
 364,
 365,
 368,
 373,
 377,
 380,
 383,
 390,
 393,
 394,
 395,
 397,
 399,
 400,
 402,
 403,
 404,
 405,
 409,
 410,
 412,
 415,
 421,
 422,
 428,
 430,
 431,
 432,
 439,
 442,
 443,
 445,
 446,
 450,
 451,
 457,

In [5]:
print(f"number of duplicate questions: {len(is_duplicate_index)}, number of non duplicate questions {len(data) - len(is_duplicate_index)}")

number of duplicate questions: 111486, number of non duplicate questions 292865


In [6]:
data.loc[is_duplicate_index[:5]]

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


Splitting out test and train q1 and q2 words

In [7]:
q1_train_words = data_train.loc[is_duplicate_index,'question1']
q2_train_words = data_train.loc[is_duplicate_index,'question2']

q1_test_words = data_test['question1']
q2_test_words = data_test['question2']
y_test = data_test['is_duplicate']


Q1 and Q2 training breakdown

In [8]:
q1_train_words[:10], q2_train_words[:10]

(5     Astrology: I am a Capricorn Sun Cap moon and c...
 7                        How can I be a good geologist?
 11          How do I read and find my YouTube comments?
 12                 What can make Physics easy to learn?
 13          What was your first sexual experience like?
 15    What would a Trump presidency mean for current...
 16                         What does manipulation mean?
 18    Why are so many Quora users posting questions ...
 20                           Why do rockets look white?
 29               How should I prepare for CA final law?
 Name: question1, dtype: object,
 5     I'm a triple Capricorn (Sun, Moon and ascendan...
 7             What should I do to be a great geologist?
 11               How can I see all my Youtube comments?
 12              How can you make physics easy to learn?
 13               What was your first sexual experience?
 15    How will a Trump presidency affect the student...
 16                        What does manipulation means

In [9]:
print(f"number of q1 train words: {len(q1_train_words)}, and number of q2 train words {len(q2_train_words)}")

number of q1 train words: 111486, and number of q2 train words 111486


Q1 and Q2 testing breakdown.

In [10]:
print(f"number of q1 test words: {len(q1_test_words)}, number of q2 test words: {len(q2_test_words)}, number of y test labels: {len(y_test)}")

number of q1 test words: 10240, number of q2 test words: 10240, number of y test labels: 10240


In [11]:
test_sentence = "How old are you"
test_words = nltk.word_tokenize(test_sentence)
test_words

['How', 'old', 'are', 'you']

In [12]:
q1_train = q1_train_words.apply(lambda x : nltk.word_tokenize(x))
q2_train = q2_train_words.apply(lambda x : nltk.word_tokenize(x))
q1_train.head(), q2_train.head()

(5     [Astrology, :, I, am, a, Capricorn, Sun, Cap, ...
 7              [How, can, I, be, a, good, geologist, ?]
 11    [How, do, I, read, and, find, my, YouTube, com...
 12       [What, can, make, Physics, easy, to, learn, ?]
 13    [What, was, your, first, sexual, experience, l...
 Name: question1, dtype: object,
 5     [I, 'm, a, triple, Capricorn, (, Sun, ,, Moon,...
 7     [What, should, I, do, to, be, a, great, geolog...
 11    [How, can, I, see, all, my, Youtube, comments, ?]
 12    [How, can, you, make, physics, easy, to, learn...
 13      [What, was, your, first, sexual, experience, ?]
 Name: question2, dtype: object)

In [13]:
q1_train = q1_train.to_numpy()
q2_train = q2_train.to_numpy()
q1_train, q2_train

(array([list(['Astrology', ':', 'I', 'am', 'a', 'Capricorn', 'Sun', 'Cap', 'moon', 'and', 'cap', 'rising', '...', 'what', 'does', 'that', 'say', 'about', 'me', '?']),
        list(['How', 'can', 'I', 'be', 'a', 'good', 'geologist', '?']),
        list(['How', 'do', 'I', 'read', 'and', 'find', 'my', 'YouTube', 'comments', '?']),
        ...,
        list(['What', 'are', 'the', 'top', '10', 'TV', 'series', 'one', 'should', 'genuinely', 'watch', '?']),
        list(['Is', 'there', 'no', 'life', 'on', 'other', 'planets', '?']),
        list(['How', 'do', 'I', 'tell', 'the', 'difference', 'between', 'infatuation', 'and', 'love', '?'])],
       dtype=object),
 array([list(['I', "'m", 'a', 'triple', 'Capricorn', '(', 'Sun', ',', 'Moon', 'and', 'ascendant', 'in', 'Capricorn', ')', 'What', 'does', 'this', 'say', 'about', 'me', '?']),
        list(['What', 'should', 'I', 'do', 'to', 'be', 'a', 'great', 'geologist', '?']),
        list(['How', 'can', 'I', 'see', 'all', 'my', 'Youtube', 'comments'

In [14]:
q1_test = q1_test_words.apply(lambda x : nltk.word_tokenize(x))
q2_test = q2_test_words.apply(lambda x : nltk.word_tokenize(x))
q1_test.head(), q2_test.head()

(300000    [How, do, I, prepare, for, interviews, for, cs...
 300001    [What, is, the, best, bicycle, to, buy, under,...
 300002    [How, do, I, become, Mutual, funds, distribute...
 300003                  [Will, this, relationship, work, ?]
 300004                [How, does, Brexit, affect, India, ?]
 Name: question1, dtype: object,
 300000    [What, is, the, best, way, to, prepare, for, c...
 300001    [Which, is, the, best, bike, in, in, dia, to, ...
 300002    [How, do, I, become, mutual, funds, distributo...
 300003    [Relationship, :, Will, this, relationship, wo...
 300004    [Will, the, GBP/AUD, be, affected, by, Brexit, ?]
 Name: question2, dtype: object)

In [15]:
q1_test = q1_test.to_numpy()
q2_test = q2_test.to_numpy()
q1_test, q2_test

(array([list(['How', 'do', 'I', 'prepare', 'for', 'interviews', 'for', 'cse', '?']),
        list(['What', 'is', 'the', 'best', 'bicycle', 'to', 'buy', 'under', '10k', '?']),
        list(['How', 'do', 'I', 'become', 'Mutual', 'funds', 'distributer', 'for', 'all', 'company', 'mutual', 'funds', '?']),
        ...,
        list(['What', 'are', 'some', 'biblical', 'examples', 'of', 'God', 'giving', 'people', 'more', 'than', 'they', 'can', 'handle', '?']),
        list(['What', 'is', 'the', 'main', 'cause', 'of', 'typhoons', '?']),
        list(['How', 'does', 'one', 'become', 'a', 'man', 'of', 'action', '?'])],
       dtype=object),
 array([list(['What', 'is', 'the', 'best', 'way', 'to', 'prepare', 'for', 'cse', '?']),
        list(['Which', 'is', 'the', 'best', 'bike', 'in', 'in', 'dia', 'to', 'buy', 'in', 'INR', '10k', '?']),
        list(['How', 'do', 'I', 'become', 'mutual', 'funds', 'distributor', 'for', 'all', 'company', 'mutual', 'funds', '?']),
        ...,
        list(['If', 'Go

In [16]:
from collections import defaultdict

vocab = defaultdict(lambda : 0)
vocab['<PAD>']=1

for idx in range(len(q1_train)):
    q = q1_train[idx] + q2_train[idx]
    for word in q:
        if word not in vocab:
            vocab[word] = len(vocab) + 1
print('The length of the vocabulary is: ', len(vocab))

The length of the vocabulary is:  36268


In [17]:
print(f"'<PAD>' index {vocab['<PAD>']}")
print(f"'Astrology' index {vocab['Astrology']}")
print(f"'What' index {vocab['What']}")
print(f"'Astronomy' index {vocab['Astronomy']}")

'<PAD>' index 1
'Astrology' index 2
'What' index 30
'Astronomy' index 0


<a name='1-2'></a>
### 1.2 - Converting a Question to a Tensor

You will now convert every question to a tensor, or an array of numbers, using your vocabulary built above.

In [18]:
q1_train_tensor = [[vocab[w] for w in words] for words in q1_train]
q2_train_tensor = [[vocab[w] for w in words] for words in q2_train]

q1_train_tensor, q2_train_tensor

([[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
  [32, 33, 4, 34, 6, 35, 36, 21],
  [32, 38, 4, 41, 11, 42, 43, 44, 45, 21],
  [30, 33, 49, 50, 51, 39, 52, 21],
  [30, 55, 56, 57, 58, 59, 60, 21],
  [30, 61, 6, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 21],
  [30, 16, 84, 64, 21],
  [86, 87, 88, 89, 90, 91, 92, 93, 17, 87, 94, 95, 72, 96, 21],
  [86, 38, 102, 103, 104, 21],
  [32, 37, 4, 107, 65, 108, 109, 110, 21],
  [30,
   87,
   116,
   117,
   118,
   65,
   119,
   120,
   6,
   121,
   17,
   122,
   123,
   124,
   78,
   125,
   21],
  [30, 130, 131, 132, 133, 61, 34, 78, 134, 135, 39, 136, 53, 137, 21],
  [32, 38, 138, 107, 65, 139, 21],
  [30, 87, 116, 142, 131, 143, 17, 33, 34, 49, 127, 144, 145, 21],
  [32, 38, 4, 49, 147, 148],
  [149, 150, 151, 35, 65, 152, 153, 154, 155, 21],
  [161, 6, 162, 163, 164, 72, 6, 165, 166, 167, 21, 168, 88, 25, 169, 21],
  [30, 156, 78, 170, 171, 172, 53, 173, 174, 175, 11, 176, 21],
  [4,
   55,
   

In [19]:
q1_test_tensor = [[vocab[w] for w in words] for words in q1_test]
q2_test_tensor = [[vocab[w] for w in words] for words in q2_test]

q1_test_tensor, q2_test_tensor

([[32, 38, 4, 107, 65, 1015, 65, 11509, 21],
  [30, 156, 78, 216, 8908, 39, 716, 286, 8317, 21],
  [32, 38, 4, 521, 1340, 735, 0, 65, 47, 1476, 1341, 735, 21],
  [161, 31, 1597, 302, 21],
  [32, 16, 1884, 77, 308, 21],
  [149,
   14536,
   4151,
   9359,
   2347,
   27422,
   2941,
   483,
   0,
   0,
   14963,
   31224,
   9359,
   2347,
   21],
  [30, 87, 116, 32497, 2111, 21],
  [283, 156, 78, 216, 1442, 1223, 4115, 21],
  [30, 156, 78, 216, 11, 1537, 232, 4282, 28, 1718, 101, 1118, 71, 21],
  [86, 38, 116, 3296, 4464, 65, 1047, 21],
  [30, 87, 78, 216, 993, 39, 1063, 28, 1114, 21],
  [30, 3591, 18117, 39, 78, 3285, 131, 633, 28133, 21],
  [30, 87, 116, 1525, 35, 2327, 4582, 860, 39, 1414, 72, 1804, 21],
  [32, 33, 0, 852, 20, 28, 2997, 4776, 21],
  [520, 78, 1227, 5655, 0, 21],
  [32, 38, 4, 1229, 2186, 11, 2174, 28, 16152, 21],
  [30, 156, 78, 233, 3374, 3371, 3372, 21],
  [168,
   15041,
   14020,
   131,
   17715,
   373,
   372,
   19186,
   0,
   13099,
   372,
   0,
   0,
   

In [20]:
train_cutoff = int(len(q1_train) * 0.8)
train_q1_final, train_q2_final = q1_train_tensor[:train_cutoff], q2_train_tensor[:train_cutoff]
val_q1, val_q2 = q1_train_tensor[train_cutoff:], q2_train_tensor[train_cutoff:]

<a name='1-3'></a>
### 1.3 - Understanding the Iterator 

Most of the time in Natural Language Processing, and AI in general we use batches when training our data sets. If you were to use stochastic gradient descent with one example at a time, it will take you forever to build a model. In this example, we show you how you can build a data generator that takes in $Q1$ and $Q2$ and returns a batch of size `batch_size`  in the following format $([q1_1, q1_2, q1_3, ...]$, $[q2_1, q2_2,q2_3, ...])$. The tuple consists of two arrays and each array has `batch_size` questions. Again, $q1_i$ and $q2_i$ are duplicates, but they are not duplicates with any other elements in the batch. 

<br>

The command ```next(data_generator)```returns the next batch. This iterator returns the data in a format that you could directly use in your model when computing the feed-forward of your algorithm. This iterator returns a pair of arrays of questions. 

<a name='ex-1'></a>
### Exercise 1 - data_generator

**Instructions:**  
Implement the data generator below. Here are some things you will need. 

- While true loop.
- if `index >= len_Q1`, set the `idx` to $0$.
- The generator should return shuffled batches of data. To achieve this without modifying the actual question lists, a list containing the indexes of the questions is created. This list can be shuffled and used to get random batches everytime the index is reset.
- Append elements of $Q1$ and $Q2$ to `input1` and `input2` respectively.
- if `len(input1) == batch_size`, determine `max_len` as the longest question in `input1` and `input2`. Ceil `max_len` to a power of $2$ (for computation purposes) using the following command:  `max_len = 2**int(np.ceil(np.log2(max_len)))`.
- Pad every question by `vocab['<PAD>']` until you get the length `max_len`.
- Use yield to return `input1, input2`. 
- Don't forget to reset `input1, input2`  to empty arrays at the end (data generator resumes from where it last left).

In [52]:
def data_generator(Q1, Q2, batch_size, pad=1, shuffle=True):
    """Generator function that yields batches of data

    Args:
        Q1 (list): List of transformed (to tensor) questions.
        Q2 (list): List of transformed (to tensor) questions.
        batch_size (int): Number of elements per batch.
        pad (int, optional): Pad character from the vocab. Defaults to 1.
        shuffle (bool, optional): If the batches should be randomnized or not. Defaults to True.
    Yields:
        tuple: Of the form (input1, input2) with types (numpy.ndarray, numpy.ndarray)
        NOTE: input1: inputs to your model [q1a, q2a, q3a, ...] i.e. (q1a,q1b) are duplicates
              input2: targets to your model [q1b, q2b,q3b, ...] i.e. (q1a,q2i) i!=a are not duplicates
    """
    input1 = []
    input2 = []
    
    idx = 0
    len_q = len(Q1)
    assert len_q == len(Q2)
    question_indexes = [*range(len_q)]
    
    if shuffle:
        rnd.shuffle(question_indexes)
    
    
    while True:
        if idx >= len_q:
            idx = 0
            
            if shuffle:
                rnd.shuffle(question_indexes)
        
        q1 = Q1[question_indexes[idx]]
        q2 = Q2[question_indexes[idx]]
        
        print(f"cur_q1 {q1}")
        print(f"cur_q2 {q2}")
        
        idx += 1
        
        input1.append(q1)
        input2.append(q2)

        if len(input1) == batch_size:
            max_q1 = max(len(q) for q in input1)
            max_q2 = max(len(q) for q in input2)
            max_len = max(max_q1, max_q2)
            max_len = 2**int(np.ceil(np.log2(max_len)))
            b1 = []
            b2 = []
            
            for q1, q2 in zip(input1, input2):
                q1=[pad if i >= len(q1) else q1[i] for i in range(max_len)]
                q2=[pad if i >= len(q2) else q2[i] for i in range(max_len)]
                b1.append(q1)
                b2.append(q2)
                print(f"Q1 {Q1}")
            
            yield np.array(b1), np.array(b2)
            input1, input2 = [], []  # reset the batches
        


In [22]:
batch_size = 2
res1, res2 = next(data_generator(train_q1_final, train_q2_final, batch_size))
print("First questions  : ",'\n', res1, '\n')
print("Second questions : ",'\n', res2)

First questions  :  
 [[   30    37     4    38   315     4   699  1166   156   497   483    20
     21     1     1     1]
 [  161  4410  4946  2076 22466    28   985   768    21     1     1     1
      1     1     1     1]] 

Second questions :  
 [[   30    37     4    38   315     4   699  1166   156   497   483    20
     21     1     1     1]
 [  161  4410  4946  2076 22466    28   985   768    21     1     1     1
      1     1     1     1]]


**Note**: The following expected output is valid only if you run the above test cell **_once_** (first time). The output will change on each execution.

If you think your implementation is correct and it is not matching the output, make sure to restart the kernel and run all the cells from the top again. 

**Expected Output:**
```CPP
First questions  :  
 [[  30   87   78  134 2132 1981   28   78  594   21    1    1    1    1
     1    1]
 [  30   55   78 3541 1460   28   56  253   21    1    1    1    1    1
     1    1]] 

Second questions :  
 [[  30  156   78  134 2132 9508   21    1    1    1    1    1    1    1
     1    1]
 [  30  156   78 3541 1460  131   56  253   21    1    1    1    1    1
     1    1]]
```
Now that you have your generator, you can just call it and it will return tensors which correspond to your questions in the Quora data set.<br>Now you can go ahead and start building your neural network. 



In [53]:
# Test your function
w4_unittest.test_data_generator(data_generator)

cur_q1 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
cur_q2 [4, 22, 6, 23, 7, 24, 8, 25, 26, 11, 27, 28, 7, 29, 30, 16, 31, 18, 19, 20, 21]
cur_q1 [32, 33, 4, 34, 6, 35, 36, 21]
cur_q2 [30, 37, 4, 38, 39, 34, 6, 40, 36, 21]
cur_q1 [32, 38, 4, 41, 11, 42, 43, 44, 45, 21]
cur_q2 [32, 33, 4, 46, 47, 43, 48, 45, 21]


IndexError: list index out of range

<a name='2'></a>
## 2 - Defining the Siamese Model

<a name='2-1'></a>
### 2.1 - Understanding Siamese Network 
A Siamese network is a neural network which uses the same weights while working in tandem on two different input vectors to compute comparable output vectors.The Siamese network you are about to implement looks like this:

<img src = "images/siamese.png" style="width:600px;height:300px;"/>

You get the question embedding, run it through an LSTM layer, normalize $v_1$ and $v_2$, and finally use a triplet loss (explained below) to get the corresponding cosine similarity for each pair of questions. As usual, you will start by importing the data set. The triplet loss makes use of a baseline (anchor) input that is compared to a positive (truthy) input and a negative (falsy) input. The distance from the baseline (anchor) input to the positive (truthy) input is minimized, and the distance from the baseline (anchor) input to the negative (falsy) input is maximized. In math equations, you are trying to maximize the following.

$$\mathcal{L}(A, P, N)=\max \left(\|\mathrm{f}(A)-\mathrm{f}(P)\|^{2}-\|\mathrm{f}(A)-\mathrm{f}(N)\|^{2}+\alpha, 0\right)$$

$A$ is the anchor input, for example $q1_1$, $P$ the duplicate input, for example, $q2_1$, and $N$ the negative input (the non duplicate question), for example $q2_2$.<br>
$\alpha$ is a margin; you can think about it as a safety net, or by how much you want to push the duplicates from the non duplicates. 
<br>

<a name='ex-2'></a>
### Exercise 2 - Siamese

**Instructions:** Implement the `Siamese` function below. You should be using all the objects explained below. 

To implement this model, you will be using `trax`. Concretely, you will be using the following functions.


- `tl.Serial`: Combinator that applies layers serially (by function composition) allows you set up the overall structure of the feedforward. [docs](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.combinators.Serial) / [source code](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/combinators.py#L26)
    - You can pass in the layers as arguments to `Serial`, separated by commas. 
    - For example: `tl.Serial(tl.Embeddings(...), tl.Mean(...), tl.Dense(...), tl.LogSoftmax(...))` 


-  `tl.Embedding`: Maps discrete tokens to vectors. It will have shape (vocabulary length X dimension of output vectors). The dimension of output vectors (also called d_feature) is the number of elements in the word embedding. [docs](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.Embedding) / [source code](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/core.py#L113)
    - `tl.Embedding(vocab_size, d_feature)`.
    - `vocab_size` is the number of unique words in the given vocabulary.
    - `d_feature` is the number of elements in the word embedding (some choices for a word embedding size range from 150 to 300, for example).


-  `tl.LSTM` The LSTM layer. It leverages another Trax layer called [`LSTMCell`](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.rnn.LSTMCell). The number of units should be specified and should match the number of elements in the word embedding. [docs](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.rnn.LSTM) / [source code](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/rnn.py#L87)
    - `tl.LSTM(n_units)` Builds an LSTM layer of n_units.
    
    
- `tl.Mean`: Computes the mean across a desired axis. Mean uses one tensor axis to form groups of values and replaces each group with the mean value of that group. [docs](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.Mean) / [source code](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/core.py#L276)
    - `tl.Mean(axis=1)` mean over columns.


- `tl.Fn` Layer with no weights that applies the function f, which should be specified using a lambda syntax. [docs](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.base.Fn) / [source doce](https://github.com/google/trax/blob/70f5364dcaf6ec11aabbd918e5f5e4b0f5bfb995/trax/layers/base.py#L576)
    - $x$ -> This is used for cosine similarity.
    - `tl.Fn('Normalize', lambda x: normalize(x))` Returns a layer with no weights that applies the function `f`
    
    
- `tl.parallel`: It is a combinator layer (like `Serial`) that applies a list of layers in parallel to its inputs. [docs](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.combinators.Parallel) / [source code](https://github.com/google/trax/blob/37aba571a89a8ad86be76a569d0ec4a46bdd8642/trax/layers/combinators.py#L152)


In [25]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: Siamese
def Siamese(vocab_size=41699, d_model=128, mode='train'):
    """Returns a Siamese model.

    Args:
        vocab_size (int, optional): Length of the vocabulary. Defaults to len(vocab).
        d_model (int, optional): Depth of the model. Defaults to 128.
        mode (str, optional): 'train', 'eval' or 'predict', predict mode is for fast inference. Defaults to 'train'.

    Returns:
        trax.layers.combinators.Parallel: A Siamese model. 
    """

    def normalize(x):  # normalizes the vectors to have L2 norm 1
        return x / fastnp.sqrt(fastnp.sum(x * x, axis=-1, keepdims=True))
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    q_processor = tl.Serial( # Processor will run on Q1 and Q2. 
        tl.Embedding(vocab_size=vocab_size,d_feature=d_model), # Embedding layer
        tl.LSTM(d_model), # LSTM layer
        tl.Mean(axis=1), # Mean over columns
        tl.Fn('Normalize', lambda x : normalize(x)), # Apply normalize function
    )  # Returns one vector of shape [batch_size, d_model]. 
    
    ### END CODE HERE ###
    
    # Run on Q1 and Q2 in parallel.
    model = tl.Parallel(q_processor, q_processor)
    return model

In [26]:
# check your model
model = Siamese()
print(model)

Parallel_in2_out2[
  Serial[
    Embedding_41699_128
    LSTM_128
    Mean
    Normalize
  ]
  Serial[
    Embedding_41699_128
    LSTM_128
    Mean
    Normalize
  ]
]


**Expected output:**  

```CPP
Parallel_in2_out2[
  Serial[
    Embedding_41699_128
    LSTM_128
    Mean
    Normalize
  ]
  Serial[
    Embedding_41699_128
    LSTM_128
    Mean
    Normalize
  ]
]
```

In [27]:
# Test your function
w4_unittest.test_Siamese(Siamese)

 All tests passed


<a name='2-2'></a>
### 2.2 - Hard Negative Mining


You will now implement the `TripletLoss`.<br>
As explained in the lecture, loss is composed of two terms. One term utilizes the mean of all the non duplicates, the second utilizes the *closest negative*. Our loss expression is then:
 
\begin{align}
 \mathcal{Loss_{1}(A,P,N)} &=\max \left( -cos(A,P)  + mean_{neg} +\alpha, 0\right) \\
 \mathcal{Loss_{2}(A,P,N)} &=\max \left( -cos(A,P)  + closest_{neg} +\alpha, 0\right) \\
\mathcal{Loss(A,P,N)} &= mean(Loss_1 + Loss_2) \\
\end{align}


Further, two sets of instructions are provided. The first set provides a brief description of the task. If that set proves insufficient, a more detailed set can be displayed.  

<a name='ex-3'></a>
### Exercise 3 - TripletLossFn

**Instructions (Brief):** Here is a list of things you should do: <br>

- As this will be run inside trax, use `fastnp.xyz` when using any `xyz` numpy function
- Use `fastnp.dot` to calculate the similarity matrix $v_1v_2^T$ of dimension `batch_size` x `batch_size`
- Take the score of the duplicates on the diagonal `fastnp.diagonal`
- Use the `trax` functions `fastnp.eye` and `fastnp.maximum` for the identity matrix and the maximum.

In [28]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: TripletLossFn
def TripletLossFn(v1, v2, margin=0.25):
    """Custom Loss function.

    Args:
        v1 (numpy.ndarray): Array with dimension (batch_size, model_dimension) associated to Q1.
        v2 (numpy.ndarray): Array with dimension (batch_size, model_dimension) associated to Q2.
        margin (float, optional): Desired margin. Defaults to 0.25.

    Returns:
        jax.interpreters.xla.DeviceArray: Triplet Loss.
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # use fastnp to take the dot product of the two batches (don't forget to transpose the second argument)
    scores = fastnp.dot(v1, v2) / (()) # pairwise cosine sim    
    # calculate new batch size
    batch_size = len(scores)
    # use fastnp to grab all postive `diagonal` entries in `scores`
    positive = None  # the positive ones (duplicates)
    # subtract `fastnp.eye(batch_size)` out of 1.0 and do element-wise multiplication with `scores`
    negative_zero_on_duplicate = None
    # use `fastnp.sum` on `negative_zero_on_duplicate` for `axis=1` and divide it by `(batch_size - 1)`
    mean_negative = None
    # create a composition of two masks: 
    # the first mask to extract the diagonal elements, 
    # the second mask to extract elements in the negative_zero_on_duplicate matrix that are larger than the elements in the diagonal 
    mask_exclude_positives = (None)|(None)
    # multiply `mask_exclude_positives` with 2.0 and subtract it out of `negative_zero_on_duplicate`
    negative_without_positive = None
    # take the row by row `max` of `negative_without_positive`. 
    # Hint: negative_without_positive.max(axis = [?])  
    closest_negative = None
    # compute `fastnp.maximum` among 0.0 and `A`
    # where A = subtract `positive` from `margin` and add `closest_negative`
    # IMPORTANT: DO NOT create an extra variable 'A'
    triplet_loss1 = None
    # compute `fastnp.maximum` among 0.0 and `B`
    # where B = subtract `positive` from `margin` and add `mean_negative`
    # IMPORTANT: DO NOT create an extra variable 'B'
    triplet_loss2 = fastnp.maximum()
    # add the two losses together and take the `fastnp.sum` of it    
    triplet_loss = fastnp.sum(triplet_loss1, triplet_loss2)    
    ### END CODE HERE ###
    
    return triplet_loss

In [29]:
v1 = np.array([[ 0.26726124,  0.53452248,  0.80178373],[-0.5178918 , -0.57543534, -0.63297887]])
v2 = np.array([[0.26726124, 0.53452248, 0.80178373],[0.5178918 , 0.57543534, 0.63297887]])
print("Triplet Loss:", TripletLossFn(v1,v2))

TypeError: dot_general requires contracting dimensions to have the same shape, got (3,) and (2,).

In [ ]:
# Test your function
w4_unittest.test_TripletLossFn(TripletLossFn)